# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [2]:
# Set the file path to import the WeatherPy_database.csv file
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Buala,SB,-8.1450,159.5921,84.65,74,79,9.66,light rain
1,1,Alyangula,AU,-13.8483,136.4192,82.74,77,100,18.97,light rain
2,2,Kaitong,CN,44.8135,123.1500,2.48,70,0,4.29,clear sky
3,3,Castro,BR,-24.7911,-50.0119,65.10,93,100,2.46,overcast clouds
4,4,Khatanga,RU,71.9667,102.5000,-7.98,97,100,11.56,light snow


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Buala,SB,-8.1450,159.5921,84.65,74,79,9.66,light rain
1,1,Alyangula,AU,-13.8483,136.4192,82.74,77,100,18.97,light rain
8,8,Vilhena,BR,-12.7406,-60.1458,70.02,97,67,4.03,broken clouds
9,9,Mataura,NZ,-46.1927,168.8643,76.24,43,100,1.23,overcast clouds
10,10,Katherine,AU,-14.4667,132.2667,79.12,89,75,11.50,broken clouds


In [5]:
len(preferred_cities_df)


236

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_df = preferred_cities_df.dropna()

# Display sample data
clean_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Buala,SB,-8.1450,159.5921,84.65,74,79,9.66,light rain
1,1,Alyangula,AU,-13.8483,136.4192,82.74,77,100,18.97,light rain
8,8,Vilhena,BR,-12.7406,-60.1458,70.02,97,67,4.03,broken clouds
9,9,Mataura,NZ,-46.1927,168.8643,76.24,43,100,1.23,overcast clouds
10,10,Katherine,AU,-14.4667,132.2667,79.12,89,75,11.50,broken clouds


In [7]:
len(clean_df)

235

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [8]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_df[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()

# Display sample data
hotel_df.head(10)



,City,Country,Max Temp,Current Description,Lat,Lng
0,Buala,SB,84.65,light rain,-8.1450,159.5921
1,Alyangula,AU,82.74,light rain,-13.8483,136.4192
8,Vilhena,BR,70.02,broken clouds,-12.7406,-60.1458
9,Mataura,NZ,76.24,overcast clouds,-46.1927,168.8643
10,Katherine,AU,79.12,broken clouds,-14.4667,132.2667
12,Kavieng,PG,84.52,overcast clouds,-2.5744,150.7967
16,Atuona,PF,78.93,few clouds,-9.8000,-139.0333
18,Beira,MZ,74.53,light rain,-19.8436,34.8389
19,Lagos,NG,77.32,scattered clouds,6.5833,3.7500
20,Hermanus,ZA,71.80,clear sky,-34.4187,19.2345


In [9]:
len(hotel_df)

235

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [10]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Buala,SB,84.65,light rain,-8.1450,159.5921,
1,Alyangula,AU,82.74,light rain,-13.8483,136.4192,
8,Vilhena,BR,70.02,broken clouds,-12.7406,-60.1458,
9,Mataura,NZ,76.24,overcast clouds,-46.1927,168.8643,
10,Katherine,AU,79.12,broken clouds,-14.4667,132.2667,
12,Kavieng,PG,84.52,overcast clouds,-2.5744,150.7967,
16,Atuona,PF,78.93,few clouds,-9.8000,-139.0333,
18,Beira,MZ,74.53,light rain,-19.8436,34.8389,
19,Lagos,NG,77.32,scattered clouds,6.5833,3.7500,
20,Hermanus,ZA,71.80,clear sky,-34.4187,19.2345,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [11]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
} 

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
       


Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
H

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [12]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Buala,SB,84.65,light rain,-8.1450,159.5921,
1,Alyangula,AU,82.74,light rain,-13.8483,136.4192,
8,Vilhena,BR,70.02,broken clouds,-12.7406,-60.1458,
9,Mataura,NZ,76.24,overcast clouds,-46.1927,168.8643,
10,Katherine,AU,79.12,broken clouds,-14.4667,132.2667,
12,Kavieng,PG,84.52,overcast clouds,-2.5744,150.7967,
16,Atuona,PF,78.93,few clouds,-9.8000,-139.0333,
18,Beira,MZ,74.53,light rain,-19.8436,34.8389,
19,Lagos,NG,77.32,scattered clouds,6.5833,3.7500,
20,Hermanus,ZA,71.80,clear sky,-34.4187,19.2345,


In [13]:
# Display sample data
len(hotel_df)


235

In [14]:
# Convert empty rows to NaN per https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe

convert_hotel_df=hotel_df["Hotel Name"].replace('',np.nan)
convert_hotel_df.head(10)

0    NaN
1    NaN
8    NaN
9    NaN
10   NaN
12   NaN
16   NaN
18   NaN
19   NaN
20   NaN
Name: Hotel Name, dtype: float64

In [15]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Buala,SB,84.65,light rain,-8.1450,159.5921,
1,Alyangula,AU,82.74,light rain,-13.8483,136.4192,
8,Vilhena,BR,70.02,broken clouds,-12.7406,-60.1458,
9,Mataura,NZ,76.24,overcast clouds,-46.1927,168.8643,
10,Katherine,AU,79.12,broken clouds,-14.4667,132.2667,
12,Kavieng,PG,84.52,overcast clouds,-2.5744,150.7967,
16,Atuona,PF,78.93,few clouds,-9.8000,-139.0333,
18,Beira,MZ,74.53,light rain,-19.8436,34.8389,
19,Lagos,NG,77.32,scattered clouds,6.5833,3.7500,
20,Hermanus,ZA,71.80,clear sky,-34.4187,19.2345,


In [16]:
# In case of an empty DataFrame, load the sample data provided
clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
clean_hotel_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
5,18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
6,20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
7,22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [17]:
# Convert empty rows to NaN per https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
len(clean_hotel_df)


352

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [18]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [19]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

Hotel Name
{Hotel Name}
City
{City}
Country
{Country}
Weather Description
{Current Description}

Max Temp
{Max Temp} °F


"""

In [34]:
# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]


['\n\nHotel Name\nPudong Holiday Hotel\nCity\nAksu\nCountry\nCN\nWeather Description\nclear sky\n\nMax Temp\n82.42 °F\n\n\n',
 '\n\nHotel Name\nINN NEW HORIZON\nCity\nTouros\nCountry\nBR\nWeather Description\nbroken clouds\n\nMax Temp\n80.6 °F\n\n\n',
 '\n\nHotel Name\nCCI Express Inn\nCity\nMorehead\nCountry\nUS\nWeather Description\nclear sky\n\nMax Temp\n75.2 °F\n\n\n',
 '\n\nHotel Name\n39 On Nile Guest House\nCity\nPort Elizabeth\nCountry\nZA\nWeather Description\nclear sky\n\nMax Temp\n84.2 °F\n\n\n',
 '\n\nHotel Name\nDurrant House Hotel\nCity\nNortham\nCountry\nGB\nWeather Description\nfew clouds\n\nMax Temp\n82.4 °F\n\n\n',
 '\n\nHotel Name\nTaj Krishna, Hyderabad\nCity\nHyderabad\nCountry\nIN\nWeather Description\nhaze\n\nMax Temp\n87.01 °F\n\n\n',
 '\n\nHotel Name\nThe Halyards Hotel\nCity\nPort Alfred\nCountry\nZA\nWeather Description\nclear sky\n\nMax Temp\n82.0 °F\n\n\n',
 '\n\nHotel Name\nVilla Enata\nCity\nAtuona\nCountry\nPF\nWeather Description\nclear sky\n\nMax Temp\

In [21]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,41.12,80.26
1,-5.20,-35.46
2,37.27,-87.18
3,-33.92,25.57
4,51.03,-4.22
5,17.38,78.47
6,-33.59,26.89
7,-9.80,-139.03
8,22.08,-159.32
9,59.70,30.79


In [35]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Display the figure containing the map
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))